<a href="https://colab.research.google.com/github/faan03/consensusProtocol/blob/main/Protocol_v3(multiplicative)_withOutCoordinator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install gensafeprime
import gensafeprime as gen
import random as random
import hashlib as hashlib

'''-------------------------------------------------------------------------------
                                    Zq
-------------------------------------------------------------------------------'''
class Zq:    
    def __init__(self,q):
        self.q = q        

    def __modulo (self,a,b):
        return a%b # residuo div entera entre a y b

    def suma(self,a,b):
        r= self.__modulo((a+b),self.q)
        return r

    def resta(self,a,b):
        r = self.__modulo (a-b,self.q)
        return r

    def producto (self,a,b):
        r= self.__modulo(a*b,self.q)
        return r 

    def division (self,a,b):
        invM_b= self.invMulti(b)
        r= self.__modulo(a*invM_b,self.q)
        return r

    def potencia (self,base, potencia):   
        r= pow(base,potencia,self.q)
        return r

    def invMulti(self,a):
        r= self.__gcdExtended(a,self.q)[1]
        return r    

    def __gcdExtended(self,a, b): 
    # Base Case 
        if a == 0 :  
            return b,0,1             
        gcd,x1,y1 = self.__gcdExtended(b%a, a) 
        # Update x and y using results of recursive 
        # call 
        x = y1 - (b//a) * x1 
        y = x1      
        return gcd,x,y        

    def getRandomElement(self):
        return random.randint(0,self.q-1)

    def getRandomNotZeroElement(self):
        return random.randint(1,self.q-1)

    def H(self,list): # toma los n parametros, los pasa a binarios los suma
        sb=''
        for l in list :
            sb= sb + bin(l)
        h=hashlib.sha256(sb.encode('utf-8')).hexdigest()        
        return self.__modulo(int(h,16),q)
        
################################################ JRNG ################################################
## https://www.scitepress.org/Papers/2021/105348/105348.pdf
'''-------------------------------------------------------------------------------
                                    Generator
-------------------------------------------------------------------------------'''
class Generator:
    def __init__(self,G,q):
        self.G=G
        self.q=q        
        self.zq = Zq(q)
        self.__getGenerator()       
    
    def __getGenerator(self):
        ##toma un elemento entre 1 y p-1        
        element= self.G.getRandomElement()
        while ((self.G.potencia (element,2)==1) or (self.G.potencia(element,self.q)!=1)):
            element= self.G.getRandomElement()
        self.g = element

    def get_g(self):             
        return self.g 
'''-------------------------------------------------------------------------------
                                    Node
-------------------------------------------------------------------------------'''
class node:
  
    def __init__(self,G,q,g,i,n,t):
        self.G = G
        self.q =q
        self.zq = Zq(q)
        self.g=g
        self.__listPublicKeys=[]
        self.i=i
        self.n=n
        self.t=t
        self.__Cis=[]
        self.__A=[]

  #----1-----
    def __generateRandomKeyPairs(self):
        self.__secretKeys = [self.zq.getRandomNotZeroElement() for j in range(0,self.n+1) ]   
        self.__publicKeys = [self.G.potencia (self.g, sk) for sk in self.__secretKeys]
        
 #-------------2-----------   
    def getPublicKeys(self):
        self.__generateRandomKeyPairs()
        return self.__publicKeys

    def catchPublicKeys (self, pks):
        self.__listPublicKeys.append(pks)  
        
# ----------------- 3 -------------------  
    def sMetod (self,j):
        if self.i>j:
            return 1
        elif self.i<j:
          return self.q-1  
    
    def __computeRp_product(self,z):    
        rpi=1                
        for j in range (0,self.t-1):
            if (self.i!=(j+1)):                
                rpi= self.G.producto (rpi,self.G.potencia(self.__listPublicKeys[j][z], self.zq.producto(self.sMetod(j+1),self.__secretKeys[z])))
        return rpi        
    
    def __computeRi(self):
        self.__Ris=[]        
        for z in range (0, self.n+1):    
            self.__Ris.append(self.__computeRp_product(z))      
        
    def getR(self):
        return self.__Ris        

#------------ 4 --------------------
    def __get_si_ci(self):
        self.c= self.zq.getRandomNotZeroElement()   
        self.s= self.zq.getRandomElement()   

#----------- 5 ------------------
    def computeCi(self, prob=1):       
        self.__computeRi()       
        self.__get_si_ci()
        Cs=[]
        #caso i=0, primera pos del Cs
        Cs.append(self.G.producto(self.G.potencia(self.g, self.s), self.__Ris[0]))
        for j in range  (1, n+1):
            Cs.append(self.G.producto (self.G.potencia(self.g, self.__e(j,prob)), self.__Ris[j])) #1...n
        return Cs 
        
    def __e(self,j,prob):
        rn= random.random()          
        #ok=0
        if rn <= prob:
            return self.zq.suma(self.zq.producto(self.c,self.zq.potencia(j,self.i)), self.s)   # cx_i+s             
        else:
            return self.zq.getRandomElement()

    def solveProof(self):      
        u = self.__A[0] #u = g^s
        u_p=1
        veces=0        
        while (u_p != u):#and (veces < 500):    
            (u_p,js)= self.recovery_u()     
            #print(f"u= {u} , u_p= {u_p} , js = {js}")         
            veces=veces+1    
        return (veces,js)               

    def recovery_u(self): 
        js=[]
        js = sorted(random.sample(range(1,self.n+1),self.t))         
        w=1        
        for i in js :       
            coef=1
            for j in js:
                if (j!=i):                                  
                    nu = self.zq.resta(0,j)
                    den = self.zq.resta(i,j)
                    coef = self.zq.producto(coef ,self.zq.division (nu,den))                                
            w = self.G.producto (w , self.G.potencia(self.__A[i],coef)) 
        return (w,js)               

    def validate_u (self, js):        
        u = self.__A[0] #u = g^s
        w=1        
        found=False
        for i in js :       
            coef=1
            for j in js:
                if (j!=i):                                  
                    nu = self.zq.resta(0,j)
                    den = self.zq.resta(i,j)
                    coef = self.zq.producto(coef ,self.zq.division (nu,den))                                
            w = self.G.producto (w , self.G.potencia(self.__A[i],coef)) 
        if (u==w):
            found=True        
        return found    


    def lambdaMetod(self,i,js):
        l=1
        for r in js:
            if (r!=i):
                nu = self.zq.resta(0,i)
                de= self.zq.resta(r,i)                
                l=self.zq.producto(l,self.zq.division(nu,de))
        return l   
    
    def catch_Ci (self, ci):
        self.__Cis.append(ci)

    def __compute_A (self,i_):
        A=1
        for i in range(0,self.t-1):
            A= self.G.producto(A,self.__Cis[i][i_])
        return A        
    
    def computeAs(self):                        
        for i_ in range (0, self.n+1): # n Ci cada uno de tamaño t-1
            self.__A.append(self.__compute_A(i_))           

    def getS(self):
        return self.s     

    def getC(self):
        return self.c

    def getI(self):
        return self.i         
   


In [43]:
p= gen.generate(100)
q=((p-1)//2)
G=Zq(p)
zq=Zq(q)
g=Generator(G,q)

t= 20
n= 26
prob=0.99

#prob^t-q1 <- un punto sea genuino
# t=n/3
# ajustar n,prob para que se logren t puntos genuinos
''' t participantes necesarios para la construccion del secreto
uno de los t participantes tomara el rol de coordinador, y los t-1 participantes
numerados de 1 a t-1 realizan lo siguiente:
'''
participants =[]

#---------------------------------------------------- JRNG ----------------------------------------------------
## se Genera secreto distibuido y los puntos (encriptados) para la contrucción del secreto
#inicia el coordinador y los t-1 participantes

for i in range(1,t):  
  participants.append(node(G,q,g.get_g(),i,n,t))

#----1---- 2 ---------
#cada participante  genera n+1 par de llaves aleatorias y las envia a los demas nodos

for pi in  participants:
    pks_i= pi.getPublicKeys()    
    for pj in participants:          
       pj.catchPublicKeys(pks_i)

#cada participante computa Ci y lo envia a los demas participantes
for pi in  participants:
    Ci= pi.computeCi(prob)   
    for pj in participants:          
       pj.catch_Ci(Ci)


participants[0].computeAs()
(veces,js) =participants[0].solveProof()

#js[0]=js[0]+1
print(veces,js)

#---------------- validar -----------
participants[1].computeAs()

found = participants[1].validate_u(js)
print(found)


15672 [2, 3, 4, 5, 6, 7, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 24, 25, 26]
True


In [ ]:
participants[0].computeAs()

sv=0
for v in range (0,100):

    (veces,js) =participants[0].solveProof()
    #js[0]=js[0]+1
    #print(veces,js)
    sv+= veces
    #---------------- validar -----------
    participants[1].computeAs()
    found = participants[1].validate_u(js)
    #print(found)

print(sv/100)    